In [1]:
import pandas as pd
from geopy.distance import geodesic
import pickle
places=pd.read_csv('Place_final.csv')
model = pickle.load(open('ranking_model', 'rb'))
places

,Unnamed: 0,Place,Rating,Pincode,Category,Attraction Type,Themes,Timings,Duration,Latitude,Longitude
0,0,Bull Temple,4.7,560004,Worship,Major,Family Religious,D M A E,1.0,12.9418,77.5687
1,1,Maha Bodhi Society Temple,4.7,560009,Worship,Major,Family Religious,M A E,1.0,12.9786,77.5807
2,2,Ragigudda Anjaneya Temple,4.7,560069,Worship,Major,Family Religious,M E,1.0,12.9147,77.5941
3,3,Sri Someshwara Swamy Temple,4.7,560008,Worship,Major,Family Religious,D M E,1.0,12.9695,77.5795
4,4,Sri Gavigangadhareshwara Swamy Temple,4.7,560019,Worship,Major,Family Religious,D M E,1.0,12.9482,77.5629
...,...,...,...,...,...,...,...,...,...,...,...
61,61,Lumbini Gardens,3.6,560024,Parks/Lakes,Major,Family Friends Couples Nature,M A E,2.0,13.0433,77.6103
62,62,Avalabetta,3.5,561209,Hills,Major,Solo Friends Adventure Couples Nature Hangout,D M A E,4.0,13.5775,77.6730
63,63,Muthyala Maduvu,3.5,562106,Waterfall,Major,Family Friends Couples Solo Nature Hangout,D M A E,2.0,12.6862,77.6651
64,64,Shivoham Shiva Temple,3.5,560017,Worship,Major,Religious Family,D M A E L,1.0,12.9578,77.6564


In [62]:
def find_dist_from_cur_loc(co,df_dist):
  co2=()
  dist=[]
  for i in range(len(df_dist)):
    co2=(df_dist['Latitude'][i],df_dist['Longitude'][i])
    dist.append(geodesic(co, co2).km)
  df_dist['Distance']=dist
  return df_dist

    
def categorising(d):
  places['Themes']
  dis=[]
  l=[]
  p=[]
  c=[]
  dur=[]
  time=[]
  lat=[]
  lon=[]
  co=(13.063262, 77.592346) 
  for i, row in places.iterrows():
      if d in row['Themes'].split(' '):
        l.append(row['Rating'])
        p.append(row['Place'])
        c.append(row['Category'])
        time.append(row['Timings'])
        lat.append(row['Latitude'])
        lon.append(row['Longitude'])
        co2=(row['Latitude'],row['Longitude'])
        dis.append(geodesic(co, co2).km)
        row['Duration']=row['Duration']+(geodesic(co, co2).km/25)
        dur.append(row['Duration'])
      
        df_new= pd.DataFrame({'Place':p,'Rating':l,'Category':c,'Distance': dis,'Duration':dur,'Timing':time, 'Latitude':lat, 'Longitude':lon })
  df11= df_new.sort_values(by=['Distance'],ascending= True).groupby(['Category']).head(10)
  df11.index= range(len(df11))

  return(df11)

def timecategory(timing,df11,f,e): 
  pl=[]
  m=[]
  ca=[]
  ds=[]
  dr=[]
  la=[]
  lo=[]
  r=[]
  for i in range(len(df11)):
    if timing in df11['Timing'][i] and df11['Category'][i]== f and e==1:
      m.append(df11['Timing'][i])
      pl.append(df11['Place'][i])
      ca.append(df11['Category'][i])
      ds.append(df11['Distance'][i])
      dr.append(df11['Duration'][i])
      la.append(df11['Latitude'][i])
      lo.append(df11['Longitude'][i])
      r.append(df11['Rating'][i])
    elif timing in df11['Timing'][i] and e==0:
      m.append(df11['Timing'][i])
      pl.append(df11['Place'][i])
      ca.append(df11['Category'][i])
      ds.append(df11['Distance'][i])
      dr.append(df11['Duration'][i])
      la.append(df11['Latitude'][i])
      lo.append(df11['Longitude'][i])
      r.append(df11['Rating'][i])
  return(pd.DataFrame({'Place':pl,'Rating':r,'Time':m,'Duration':dr,'Distance':ds,'Category':ca,'Latitude':la, 'Longitude':lo}))


def generate_ranking(dfr):
  ranking=[]
  for i in range(len(dfr)):
    ranking.append(float(model.predict([[dfr['Rating'][i],dfr['Distance'][i],dfr['Duration'][i]]])))
  dfr['ranking']=ranking
  # print(dfr)
  return dfr

In [3]:
entered_theme=input("Enter the Theme: ")
dfs= categorising(entered_theme)
if entered_theme =='Family' or entered_theme =='Solo' or entered_theme =='Couples' or entered_theme =='Friends':
  f=input("Enter the Sub Category like Nature, Adventure, Religious: ")
  e=1
else: 
  e=0
  f=entered_theme
dfs

,Place,Rating,Category,Distance,Duration,Timing,Latitude,Longitude
0,"Fun World , Bangalore",4.0,Amusement Park,6.248141,1.249926,M A E,13.0068,77.5910
1,"Wonder La water park, Jadenahalli",4.5,Amusement Park,32.537658,7.301506,A E,12.8360,77.4020
2,Nandi Hill,4.0,Hills,35.383835,3.415353,D M A E,13.3700,77.6848
3,Savandurga,4.6,Hills,35.868429,3.434737,D M A E,12.9160,77.2978
4,Channarayaswamy betta,4.7,Hills,36.684970,4.467399,D,13.3866,77.6674
5,Innovative Film City,3.3,Amusement Park,37.047340,7.481894,M A E,12.7767,77.4157
6,Kalawara betta (Skandagiri),4.4,Hills,40.373694,6.614948,D M,13.4173,77.6827
7,Ramadevarabetta,4.5,Hills,46.200351,3.848014,M A E,12.7563,77.3037
8,Devarayanadurga,4.0,Hills,53.494064,5.139763,M A,13.3741,77.2143
9,"Antaragange, Hogari",4.4,Hills,55.632855,6.225314,D A E,13.1494,78.0978


In [175]:
st='DMAENL'

visited=[]

start_time=input("Enter the time You want to Start(D M A E):")
end_time=input("Enter the time You want to End(A E N L):")

start_index= st.find(start_time)
end_index= st.find(end_time)
a= st[start_index: end_index+1]

for x in a:
    globals()['df%s' % x] = timecategory(x,dfs,f,e)

a1= globals()['df%s' % a[0]]
a1= generate_ranking(a1)  
sorted= a1.sort_values(by=['ranking']).reset_index(drop=True)
# co=(sorted.iloc[0][6],sorted.iloc[0][7])
# print(co)
visited.append(sorted.iloc[0])
df_visited=pd.DataFrame(visited)
for i in range(1,len(a)):
    a1= globals()['df%s' % a[i]]
    try:
        co=(df_visited['Latitude'][i-1],df_visited['Longitude'][i-1])
    except:
        break
    a1=find_dist_from_cur_loc(co,a1)
    a1= generate_ranking(a1)  
    sorted= a1.sort_values(by=['ranking']).reset_index(drop=True)
    j=0
    for j in range(len(sorted)):
        if sorted.iloc[j][0] not in set(df_visited['Place']) and sorted.iloc[j][5] not in set(df_visited['Category']):
            df_visited=df_visited.append(sorted.iloc[j],ignore_index = True)

            break
        else:
            j+=1
df_visited

,Place,Rating,Time,Duration,Distance,Category,Latitude,Longitude,ranking
0,"Fun World , Bangalore",4.0,M A E,1.249926,6.248141,Amusement Park,13.0068,77.5910,2.710173
1,Savandurga,4.6,D M A E,3.434737,33.360906,Hills,12.9160,77.2978,3.427876
2,Bheemeshwari,4.5,M A E,6.592720,66.809962,Parks/Lakes,12.3124,77.2778,8.567877
